# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.basics import patch
from __future__ import annotations
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export 
from __future__ import annotations
from torch import nn
import inspect 

In [ ]:
#| export
def init_cnn(module # either nn.Linear or nn.Conv2d
            ):
    "Initialize weights for neural net layer with xavier uniform initializer."

    if type(module) == nn.Linear or type(module) == nn.Conv2d:
        nn.init.xavier_uniform_(module.weight)

In [ ]:
#| export
class HyperParameters:
    "Saves all non-ignored arguments in a class' __init__ method as attributes."
    
    def save_hyperparameters(self, 
                             ignore = [] #  list of argument names (string) to ignore when calling `setattr`
                            ):

        # Get the next outer frame object (this frame’s caller) -
        # `__init__` method.
        frame = inspect.currentframe().f_back
        # Get information about arguments passed into a particular frame.
        _, _, _, local_vars = inspect.getargvalues(frame)

        self.hparams = {
            k: v for k, v in local_vars.items()
            # ignore 'self', any variable in `ignore` list and
            # that not starts with the underscore.
            if k not in set(ignore+['self']) and not k.startswith('_')}
        for k, v in self.hparams.items():
            # set attributes of a class
            setattr(self, k, v)

In [ ]:
show_doc(HyperParameters.save_hyperparameters)

---

[source](https://github.com/Iamalos/dlnb/blob/main/dlnb/core.py#L30){target="_blank" style="float:right; font-size:smaller"}

### HyperParameters.save_hyperparameters

>      HyperParameters.save_hyperparameters (ignore=[])

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| ignore | list | [] | list of argument names (string) to ignore when calling `setattr` |

In [ ]:
class Person(HyperParameters):
    def __init__(self, name, age, sex):
        self.save_hyperparameters(ignore=['sex'])
        
jack = Person("Billy", 25, 'male')

Saves all arguments in a class `__init__` method as class attributes.

In [ ]:
test_eq(jack.name, "Billy")

Ignores `self`, any variables in `ignore` list and that starts with
the underscore.

In [ ]:
test_fail(lambda: jack.sex, contains="'Person' object has no attribute 'sex'")

## Module

In [ ]:
#| export
class Module(nn.Module, HyperParameters):
    "The base class for all the models in the Diving into DL course"

In [ ]:
@patch
def __init__(
    self: Module,
    plot_train_per_epoch: int = 2, # number of training plot updates per one epoch.
    plot_valid_per_epoch: int = 1 # number of validation plot updates per one epoch.
):
    super().__init__()
    self.save_hyperparameters()
    self.board = ProgressBoard() # ProgressBoard for plotting data.

In [ ]:
@patch
def loss(self: Module, 
         y_hat: Tensor, # predicted y values
         y: Tensor): # real y values
    "Calculate loss between fitted values and observed values."
    raise NotImplementedError

In [ ]:
@patch
def forward(self: Module, 
            X: Tensor # input tensor
           ):
    "Make a forward pass on the data."
    # make sure class has `net` attribute
    assert hasattr(self, 'net'), 'Neural network is not defined.'
    return self.net(X)    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()